In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.spatial import cKDTree
from sklearn.neighbors import KDTree

In [3]:
import scipy
scipy.__version__

'1.4.1'

Your task is to modify `KNNClassifier` class from your practice in class. The `KNNClassifier` class with empty methods is provided below. Please, modify it to do all tasks.

In [4]:
class KNNClassifier(object):
    
    def __init__(self, max_dist=1., n_neighbors=3, use_kd_tree=False, use_weights=False):
        """
        This is a constructor of the class. 
        Here you can define parameters (max_dist) of the class and 
        attributes, that are visible within all methods of the class.
        
        Parameters
        ----------
        max_dist : float
            Maximum distance between an object and its neighbors.
        """
        
        # Make this parameter visible in all methods of the class
        self.n_neighbors = n_neighbors
        self.max_dist = max_dist
        self.use_kd_tree = use_kd_tree
        self.use_weights = use_weights
                
    
    def fit(self, X, y):
        """
        This method trains the KNN classifier. 
        Actualy, the KNN classifier has no training procedure.
        It just remembers data (X, y) that will be used for predictions.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
        y : numpy.array, shape = (n_objects)
            1D array with the object labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """
        
        self.X = X
        self.y = y
        
    def predict(self, X):
        """
        This methods performs labels prediction for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted : numpy.array, shape = (n_objects)
            1D array with predicted labels. 
            For the classification labels are integers in {0, 1, 2, ...}.
        """
        
        y_predicted = []
        self.labels_count = []
        if self.max_dist:
            for x_sample in X:
                if self.use_kd_tree:
                    index_nei = cKDTree(self.X, leafsize=30).query_ball_point(x_sample, self.max_dist)
                    classes = self.y[index_nei]
                    objects = self.X[index_nei]
                    weight = np.array([1] * len(classes))
                    if self.use_weights:
                        distance = np.apply_along_axis(lambda dataset_row: 
                                        self._distance_between_dataset(dataset_row, x_sample), 1, objects)
                        weight = distance[np.where(distance < self.max_dist)]**(-1)
                    self.result = {}
                    for label in np.unique(self.y):
                        self.result[label] = weight[classes == label].sum()
                    self.labels_count.append(self.result)
                    y_predicted.append(Counter(self.result).most_common()[0][0])
                else:
                    distance = np.apply_along_axis(lambda dataset_row: 
                                        self._distance_between_dataset(dataset_row, x_sample), 1, self.X)
                    classes = self.y[np.where(distance < self.max_dist)]
                    weight = np.array([1] * len(np.where(distance < self.max_dist)[0]))
                    if self.use_weights:
                        weight = distance[np.where(distance < self.max_dist)]**(-1)
                    self.result = {}
                    for label in np.unique(self.y):
                        self.result[label] = weight[classes == label].sum()
                    self.labels_count.append(self.result)
                    y_predicted.append(Counter(self.result).most_common()[0][0])

        else:
            self.n_neighbors = 3 if self.n_neighbors is None else self.n_neighbors
            for x_sample in X:
                if self.use_kd_tree:
                        index_nei = cKDTree(self.X, leafsize=30).query(x_sample, self.n_neighbors)[1]
                        classes = self.y[index_nei]
                        objects = self.X[index_nei]
                        weight = np.array([1] * len(classes))
                        if self.use_weights:
                            distance = np.apply_along_axis(lambda dataset_row: 
                                            self._distance_between_dataset(dataset_row, x_sample), 1, objects)
                            weight = distance**(-1)
                        self.result = {}
                        for label in np.unique(self.y):
                            self.result[label] = weight[classes == label].sum()
                        self.labels_count.append(self.result)
                        y_predicted.append(Counter(self.result).most_common()[0][0])
                else:
                    distance = np.apply_along_axis(lambda dataset_row: 
                                        self._distance_between_dataset(dataset_row, x_sample), 1, self.X)
                    classes = self.y[distance.argsort()[:self.n_neighbors]]
                    weight = np.array([1] * self.n_neighbors)
                    if self.use_weights:
                        weight = distance[distance.argsort()[:self.n_neighbors]]**(-1)
                    self.result = {}
                    for label in np.unique(self.y):
                        self.result[label] = weight[classes == label].sum()
                    self.labels_count.append(self.result)
                    y_predicted.append(Counter(self.result).most_common()[0][0])                    
            
        return np.array(y_predicted)
    
    def _distance_between_dataset(self, x_dataset, x_new, dtype='euclidean'):
        """
        dtype : distance type('euclidean', 'manhatten')
        """
        if dtype == 'euclidean':
            distance = lambda a: np.sqrt(np.sum((a - x_new)**2))
        if dtype == 'manhatten':
            distance = lambda a: np.abs(np.sum((a - x_new)**2))
        if dtype == 'cosine':
            distance = lambda a: 1 - (a.T @ x_new) / (a.T @ a * x_new.T @ x_new)
        return distance(x_dataset)
        
    def predict_proba(self, X):
        """
        This methods performs prediction of probabilities of each class for new objects.
        
        Parameters
        ----------
        X : numpy.array, shape = (n_objects, n_features)
            Matrix of objects that are described by their input features.
            
        Returns
        -------
        y_predicted_proba : numpy.array, shape = (n_objects, n_classes)
            2D array with predicted probabilities of each class. 
            Example:
                y_predicted_proba = [[0.1, 0.9],
                                     [0.8, 0.2], 
                                     [0.0, 1.0], 
                                     ...]
        """
        
        # Create an empty list for predictions
        y_predicted_proba = []
        self.predict(X)
        for label in self.labels_count:
            y_predicted_proba.append([label[0]/(label[1] + label[0]), 
                                      label[1]/(label[1] + label[0])])
            
        return np.array(y_predicted_proba) # return numpy.array

### Task 1 (1 point) <br/>
Create a matrix of object features `X` and vector of labels `y` for N=1000 objects using `sklearn.datasets.make_moons()` function from scikit-learn library. Also, set up random state in the function `random_state=42` and `noise=0.2`. To open the function description use `Shift` + `Tab` .

In [5]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=1000, random_state=42, noise=0.2)


### Check your solution
ans = np.array([[-0.112,  0.52 ],
                [ 1.143, -0.343]])
assert np.array_equal(np.round(X[:2], 3), ans), ('Check your solution.')

### Task 2 (1 point) <br/>

Split the sample into train and test samples using `sklearn.model_selection.train_test_split()` function from scikit-learn library. Use `random_state = 42` and `test_size = 0.5`.

In [6]:
### Your code here
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.5)


### Check your solution
ans = np.array([[ 0.77 , -0.289],
                [ 0.239,  1.041]])
assert np.array_equal(np.round(X_train[:2], 3), ans), ('Check your solution.')

### Task 3 (2 points) <br/>

Modify class `KNNClassifier` above and implement `predict()` method that uses **max_dist** parameter to select neighbors like it's shown in the second figure (radius search). If there is no any object within **max_dist**, make decision based on the closest neighbor.

<img src="img/knn2.png" width="600">

In [7]:
# Create a class object
knn = KNNClassifier(max_dist=0.5)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)



### Check your solution
assert accuracy_test == 0.964, ('Check your solution.')

CPU times: user 3.24 s, sys: 25.2 ms, total: 3.27 s
Wall time: 3.28 s
Test accuracy of KNN classifier:  0.964


### Task 4 (2 points) <br/>

There are an algorithm [kd-tree](https://en.wikipedia.org/wiki/K-d_tree) that helps to find neighbors faster. Using [scipy.spatial.cKDTree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.cKDTree.html#scipy.spatial.cKDTree) function modify you classifier to speed up **predict** method. Use `leafsize=30` in `KDTree`. Similar to `max_dist` option, add option `use_kd_tree = True/False` to your classifier.

In [8]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)

### Check your solution
assert accuracy_test == 0.964, ('Check your solution.')

CPU times: user 153 ms, sys: 89 µs, total: 153 ms
Wall time: 154 ms
Test accuracy of KNN classifier:  0.964


### Task 5 (3 points) <br/>

Now modify the **predict** method to provide prediction with neighbors weights.

<img src="img/wv1.png">

<img src="img/wv2.png">

We propose you to use the following weights:

$$
w_{i} = \frac{1}{\rho(x, x_{i})}
$$

Similar to `max_dist` option, add option `use_weights = True/False` to your classifier.

In [9]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True, use_weights=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict = knn.predict(X_test) # measure time for prediction

# Import accuracy_score function
from sklearn.metrics import accuracy_score
# Calculate accuracy for the test sample
accuracy_test = accuracy_score(y_test, y_test_predict)
print("Test accuracy of KNN classifier: ", accuracy_test)



### Check your solution
assert accuracy_test == 0.968, ('Check your solution.')

CPU times: user 569 ms, sys: 99 µs, total: 569 ms
Wall time: 569 ms
Test accuracy of KNN classifier:  0.968


### Task 6 (3 points) <br/>

Develop **predict_proba** method of the classifier. For each object this method returns probability that the object belongs to each of the classes. 

For each object $x$ probability for each class is defined as:

$$
p_{c}(x) = \frac{g_{c}(x)}{\sum_{i=1}^{C} g_{i}(x)}
$$

where $C$ is number of classes.

Then, the object has a vector of probabilities:

$$
p(x) = (p_{1}(x), p_{2}(x), ..., p_{C}(x))
$$

Use neighbors weights as in Task 5.

In [9]:
# Create a class object
knn = KNNClassifier(max_dist=0.5, use_kd_tree=True, use_weights=True)

# Train the classifier
knn.fit(X_train, y_train)

# Make prediction using the trained classifier
%time y_test_predict_proba = knn.predict_proba(X_test) # measure time for prediction

# Example of the output
y_test_predict_proba[:10, :] # the first 10 rows



### Check your solution
ans = np.array([[0.046, 0.954],
                [0.962, 0.038]])
assert np.array_equal(np.round(y_test_predict_proba[:2], 3), ans), ('Check your solution.')

CPU times: user 581 ms, sys: 3.82 ms, total: 585 ms
Wall time: 585 ms
